In [ ]:
from google.colab import drive

drive.mount('/content/drive/')
%cd drive/MyDrive/Multistep-reasoning

Mounted at /content/drive/
/content/drive/MyDrive/Multistep-reasoning


In [ ]:
# install required dependencies
%pip install -r entailment_bank/requirements.txt
%pip install torch

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-y1blu8ep
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-y1blu8ep
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.6 MB/s eta 0:00:00
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456763 sha256=762c16fbfffe44e340e858beaf7a9e4d6074f6d8eda1f86e9376408f049437d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-ba_h_81w/wheels/64/f4/2c/509a6c31b8ebde891a81029fd94f199b1b92f0e7cfc20d417a
Successfully built BLEURT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.4 MB/s eta 0:00:00
     

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# Load the model
from entailment_bank.utils.nlp_agent import MultiAngleModel, NlpAgent
ew_model = MultiAngleModel(model_path="allenai/entailer-large", cuda_devices=None)
prover = NlpAgent(model=ew_model, default_outputs="proof")
entail_verifier = NlpAgent(model=ew_model, default_outputs=["implied"], default_options={"explicit_outputs": ['true', 'false']})
hyp_verifier = NlpAgent(model=ew_model, default_outputs=["valid"], default_options={"explicit_outputs": ['true', 'false']})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Try to prove a hypothesis
hyp = "The leaves of plant benefit from sun"
proof = prover({"hypothesis": hyp})
premises = [x.strip() for x in proof.split("[PREMISE]") if x.strip()]

In [ ]:
hyp, premises

('The leaves of plant benefit from sun',
 ['A leaf absorbs sunlight to perform photosynthesis.',
  'Photosynthesis produces a large amount of energy for the plant.'])

In [ ]:

# Does the model think the reasoning is good? Yes:
print(entail_verifier({"hypothesis": hyp, "proof": proof}))
# {'implied': 'true', 'output_prob': 0.9999831914921629}

# Does the model believe the original hypothesis? No:
print(hyp_verifier({"hypothesis": hyp}))
# {'valid': 'false', 'output_prob': 0.9711676239967346}

# Does the model believe the premises in the proof? Yes:
print(hyp_verifier({"hypothesis": premises[0]}))
# {'valid': 'true', 'output_prob': 0.9990471005439758}

print(hyp_verifier({"hypothesis": premises[1]}))
# {'valid': 'true', 'output_prob': 0.9997676014900208}

{'implied': 'true', 'output_prob': 0.9999829530736122}
{'valid': 'true', 'output_prob': 0.9980781078338623}
{'valid': 'true', 'output_prob': 0.9999988079071045}
{'valid': 'true', 'output_prob': 0.9125959873199463}


In [ ]:
def generate_entailment_tree(hyp, model, prover, entail_verifier, hyp_verifier, visited, max_depth=3):
    """
    Entailer's backchaining algorithm for searching for the best proof tree(H) and score s(H) for a hypothesis H.
    """
    tree = {}

    if max_depth == 0:
        return 0, {}

    visited[hyp] = True

    sd_H = hyp_verifier({"hypothesis":hyp})['output_prob']
    cd_H = max(sd_H, 1-sd_H)

    P = one_step(hyp, model, prover, entail_verifier, hyp_verifier, k=5)

    if P is None:
        return 0,{}

    premises = [x.strip() for x in P.split("[PREMISE]") if x.strip()]

    entail_score = entail_verifier({"hypothesis": hyp, "proof":P})['output_prob']

    p_tree = [None]*len(premises)
    p_score = [None]*len(premises)

    if entail_score>cd_H:
        sr_H = entail_score
        print("Visited keys: ", visited.keys())
        for i in range(len(premises)):
            print(premises[i])
            if premises[i] not in visited.keys():
                p_score[i], p_tree[i] = generate_entailment_tree(premises[i], model, prover, entail_verifier, hyp_verifier, visited, max_depth-1)
                sr_H *= p_score[i]
    else:
        sr_H = 0

    cr_H = sr_H

    # if reasoning confidence is higher we expand the node
    print(f"For HYP: {hyp}")
    print(premises)
    print(cr_H, cd_H)
    if cr_H > cd_H:
        tree_score = sr_H
        for i in range(len(premises)):
            tree[premises[i]] = p_tree[i]
    else:
        tree_score = sd_H

    # print(tree)

    return tree_score, tree

def one_step(hyp, model, prover, entail_verfier, hyp_verifier, k=5):
    """
    Finds the best set of premises P which entail the hypothesis H
    """
    premises_dict = {}
    for i in range(k):
        proof = prover({"hypothesis": hyp})
        premises = [x.strip() for x in proof.split("[PREMISE]") if x.strip()]

        premise_score = [None]*len(premises)

        for j in range(len(premises)):
            ver_res = hyp_verifier({"hypothesis": premises[j]})

            premise_score[j] = ver_res['output_prob'] if ver_res['valid']=='true' else 1-ver_res['output_prob']

        entail_res = entail_verfier({"hypothesis": hyp, "proof": proof})
        entail_score = entail_res['output_prob'] if entail_res['implied']=='true' else 1-entail_res['output_prob']

        if min(premise_score)<0.5 or entail_score<0.5:
            continue

        score = entail_score
        for j in range(len(premises)):
            score *= premise_score[j]

        premises_dict[proof] = score

    if len(premises_dict.keys())>0:
        best_premises = max(premises_dict, key=lambda x: premises_dict[x])
    else:
        best_premises = None

    # print("Best Premises: ", best_premises)

    return best_premises

In [ ]:
hyp = "The leaves of a plant benefit from sun"
score, tree = generate_entailment_tree(hyp, ew_model, prover, entail_verifier, hyp_verifier, {}, 2)

Visited keys:  dict_keys(['The leaves of a plant benefit from sun'])
A leaf absorbs sunlight to perform photosynthesis.
For HYP: A leaf absorbs sunlight to perform photosynthesis.
['A leaf requires sunlight to perform photosynthesis.', 'If something is required for a process then that something can be absorbed for that process.']
0 0.9999988079071045
Photosynthesis helps the leaves of a plant absorb solar energy.
For HYP: The leaves of a plant benefit from sun
['A leaf absorbs sunlight to perform photosynthesis.', 'Photosynthesis helps the leaves of a plant absorb solar energy.']
0.0 0.9992702603340149


In [ ]:
print(tree)